In [1]:
"""──────────────────────────────────────────────────────────────────────────┐
│ Loading necessary libraries to build and train model                       │
└──────────────────────────────────────────────────────────────────────────"""
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
import os,sys,gc
import numpy as np
import pickle
import proplot as plot
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
import pandas as pd
import seaborn as sns
import glob,gc
sys.path.insert(1, '/work/FAC/FGSE/IDYST/tbeucler/default/freddy0218/TCGphy/2020_TC_CRF/dev/freddy0218/scikit/')
from tools import derive_var,read_and_proc
from copy import deepcopy
%matplotlib inline
plot.rc.update({'figure.facecolor':'w','axes.labelweight':'ultralight',
                'tick.labelweight':'ultralight','gridminor.linestyle':'--','title.weight':'normal','linewidth':0.5})
import ast
import xarray as xr
import SHIPS_func

In [2]:
data2="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/Data/ECMWF/ERA5_25kmx3hr/"
path="/work/FAC/FGSE/IDYST/tbeucler/default/saranya/causal/besttracks/"

finalstormlist = pd.read_csv('./store/stormlist_withMPI.csv')

In [4]:
era5 = SHIPS_func.readyear_automatic(data2,2000)

In [5]:
basin = [ast.literal_eval(obj)[1][:2] for obj in finalstormlist['id'].values]
year  = [int(ast.literal_eval(obj)[0][-4:]) for obj in finalstormlist['id'].values]
stormname = [obj.split('_')[1] for obj in finalstormlist['name'].values]
time_before_tcg	 = [int(obj/6) for obj in finalstormlist['time_before_tcg'].values]

In [ ]:
track = sorted(glob.glob(path+f'wp/wp_{2012}.csv'))
tracksDF = pd.read_csv(track[0])
make_timeseries_onestorm(tracksDF,stormname,'WPAC')

In [12]:
distances = []
for i in tqdm(range(len(year))):
    try:
        _,TClocxy,landlocxy = SHIPS_func.get_closest_land_xy(path,era5,basin[i],year[i],stormname[i],time_before_tcg[i])
        distance = SHIPS_func.get_distance_km(era5,TClocxy,landlocxy)
        distances.append(distance)
    except Exception as text:
        print(text)
        distances.append(np.nan)
        continue

  0%|          | 0/263 [00:00<?, ?it/s]

index 4 is out of bounds for axis 0 with size 0
index 15 is out of bounds for axis 0 with size 0
index 4 is out of bounds for axis 0 with size 0


In [13]:
finalstormlist['distance'] = distances

In [14]:
finalstormlist[finalstormlist['name']=='2012_SON-TINH']

,Unnamed: 0,index,name,id,mean_shear,time_before_tcg,mean_shear_hu,time_before_hu,lat,E_MPI,distance
78,78,4,2012_SON-TINH,"['wp242012', 'WP_24']",3.979401,36,6.277938,72,9.725,76.127868,61.184965


In [15]:
finallist = finalstormlist.drop(columns=['Unnamed: 0'])
finallist.to_csv('./store/stormlist_withMPIdistance.csv')

# Test

In [69]:
track = sorted(glob.glob(path+f'ep/EP_{2020}*.csv'))
tracksDF = pd.read_csv(track[0])
#stormloc = make_timeseries_onestorm(tracksDF,stormname,'EPAC')[int(time_before_tcg)]

In [68]:
TCloc,TClocxy,landlocxy = SHIPS_func.get_closest_land_xy(path,era5,basin[221],year[221],stormname[221],time_before_tcg[221])

IndexError: index 2 is out of bounds for axis 0 with size 0

In [10]:
probTCs = []
for num in (list(np.argwhere(np.isnan(np.asarray(distances))))):
    probTCs.append((basin[int(num)],year[int(num)],stormname[int(num)],num))

read_and_proc.save_to_pickle('./store/probTClist.pkl',probTCs)